In [ ]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.realpath('../'))

import LDAQ

In [ ]:
# ------------------------------------------
# Create simulated data acquisition sources:
# ------------------------------------------
acq_simulated_data = LDAQ.simulator.SimulatedAcquisition(acquisition_name='sim_data')

In [ ]:
# define simulated data using numpy array (array will be looped over):
fs = 10000
t = np.arange(fs*2)
x = np.sin(2*np.pi*50*t/fs) + np.random.randn(len(t))*0.2
y = np.sin(2*np.pi*100*t/fs + np.pi/2)
simulated_data = np.array([x, y]).T

# define simulated data using a function:
def simulate_signal(t, f1, f2):
    """
    first argument must be time.
    If multi_processing is True, then the function must be pickable and 
    all libraries must be imported inside the function.
    """
    import numpy as np
    sig1 = np.sin(2*np.pi*f1*t) + np.random.rand(len(t))*0.3
    sig2 = np.cos(2*np.pi*f2*t) + np.random.rand(len(t))*0.3
    return np.array([sig1, sig2]).T

In [ ]:
# set simulated data (first argument is the function or array):
acq_simulated_data.set_simulated_data(simulated_data, channel_names=["ch1", "ch2"], sample_rate=fs, args=(84, 120)) # use array
# acq_simulated_data.set_simulated_data(simulate_signal, channel_names=["ch1", "ch2"], sample_rate=fs, args=(84, 120)) # use function

In [ ]:
# ------------------------------------------
# create simulated video acquisition source:
# ------------------------------------------
acq_simulated_video = LDAQ.simulator.SimulatedAcquisition(acquisition_name='sim_video')

In [ ]:
# define simulated video using numpy array:
fs = 30
simulated_video = np.random.randn(1000, 100, 100)

# define simulated video using a function:
def simulate_video(t):
    """
    first argument must be time.
    If multi_processing is True, then the function must be pickable and 
    all libraries must be imported inside the function.
    """
    import numpy as np
    N = t.shape[0]	
    return np.random.randn(N, 100, 100)

In [ ]:
acq_simulated_video.set_simulated_video(simulated_video, channel_name_video='cam', sample_rate=fs) # use array
#acq_simulated_video.set_simulated_video(simulate_video, channel_name_video='cam', sample_rate=fs) # use function

In [ ]:
# ----------------------------
# Create visulaization object:
# ----------------------------

vis = LDAQ.Visualization(sequential_plot_updates=False)
vis.add_lines((0,0), source='sim_data', channels=["ch1", "ch2"], refresh_rate=50)
vis.add_lines((1,0), source='sim_data', channels=["ch1", "ch2"], refresh_rate=500, function="fft")

vis.add_image(source="sim_video", channel="cam", refresh_rate=30)

vis.config_subplot((0,0), t_span=0.04, ylim=(-1.1, 1.1))
vis.config_subplot((1,0), t_span=0.5, ylim=(0, 1.1), xlim=(0, 500))

In [ ]:
ldaq = LDAQ.Core(acquisitions=[acq_simulated_data, acq_simulated_video], visualization=vis)
ldaq.run(10.) # run for 10 seconds
#ldaq.run(60., save_interval=4.0, save_channels=["ch1"],) # run for 60 seconds and save data every 4 seconds, save only channel "ch1"

In [ ]:
ldaq.get_measurement_dict()